<a href="https://colab.research.google.com/github/rdezwart/Computer-Vision-Project/blob/dev/FoodClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Classification Using YOLO

## Required Libraries


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import cv2
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/107.7 GB disk)


In [5]:
from ultralytics import YOLO

## Training YOLO

In [34]:
model = YOLO('yolov8s-cls.pt')

Ignore this config file, turns out it's not needed for classification.

In [35]:
%%writefile config.yaml
path: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db
train: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/train
test: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/test
val: /content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db/eval

# Classes
nc: 11 # replace based on your dataset's number of classes

# Class names
# replace all class names with your own classes' names
names:
  0: bread
  1: dairy
  2: dessert
  3: egg
  4: fried
  5: meat
  6: noodles
  7: produce
  8: rice
  9: seafood
  10: soup

Overwriting config.yaml


In [ ]:
model.train(data="/content/drive/MyDrive/2024/IAT_481/Datasets/Food_Images/food_db",epochs=1,batch=-1)